In [ ]:
import numpy as np
import tensorflow as tf

# %matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as colors 

import ipysh
import Hunch_utils  as Htls
import Hunch_lsplot as Hplt
import Hunch_tSNEplot as Hsne

%aimport models.base
%aimport models.AEFIT3
%aimport models.CAEFIT
%aimport models.CAEFIT1

In [ ]:
qsh = Htls.QSH_Dataset()
import os
file = ipysh.abs_builddir+'/te_db_r15.npy'
if os.path.isfile(file):
    qsh.load(file)
else:
    qsh.load(ipysh.abs_builddir+'/te_db_1.npy')    
    qsh.rebalance_prel(15)
    qsh.save(ipysh.abs_builddir+'/te_db_r15.npy')

In [ ]:
qsh.shuffle()
# qsh.filter_number_set(15)
qsh.clean_up_poorcurves(5)
qsh.dim = 15
qsh.set_null(np.nan)
qsh.set_normal_positive()
qsh.unbias_mean(0.5, 'te')
qsh.set_normal_positive()

print("QSH rebalanced 15 points size: ", len(qsh))

In [ ]:
qsh.set_null(np.nan)
qsh.clip_values(0.1,0.6)
qsh.set_normal_positive()
qsh.set_null(np.nan)

## t-SNE Plot

In [ ]:
qsh.set_null(-1)
sne = Htls.tSNE()
# sne.perplexity = 50.
sne.random = 42

ds = qsh[range(0,4000)]
ds = np.concatenate([ds['prel'],ds['te']], axis=1)
# ds = np.array(list(zip(ds['prel'],ds['te'])))
sne(ds)

In [ ]:
from bokeh.io import show, output_notebook
output_notebook()

p = Hsne.tSNE_PlotBokeh()
p.set_model(sne)
p.set_data(qsh, 1000)
p.plot(notebook_url='http://172.17.0.2:8888')

## VAE reconstruction - beta

In [ ]:
from models import AEFIT2
q_vae = AEFIT2(latent_dim=2, feature_dim=30, dprate=0.4, scale=4, beta=500.)


im1 = tf.keras.utils.plot_model(q_vae.inference_net, show_shapes=True, rankdir='LR')
im2 = tf.keras.utils.plot_model(q_vae.generative_net, show_shapes=True, rankdir='LR')
from IPython.display import display
display(im1)
display(im2)

In [ ]:
models.test_dummy(q_vae, qsh, batch=200, epoch=10, loss_factor=1e-3)

In [ ]:
# q_vae.save('step6_beta_r15_2')

In [ ]:
q_vae.load('step6_beta_r15_2')

In [ ]:
from bokeh.io import show, output_notebook
output_notebook()

p = Hplt.LSPlotBokeh()
p.set_model(q_vae)
p.set_data(qsh, counts=8000)
p.plot(notebook_url='http://172.17.0.2:8888')

In [ ]:
# extract few points curve
import copy

qsh2 = Htls.QSH_Dataset()
qsh2.load(ipysh.abs_builddir+'/te_db_r15.npy')
qsh2.dim = 15
qsh2.set_null(np.nan)
qsh2.set_normal_positive()
qsh2.unbias_mean(0.5, 'te')
qsh2.set_normal_positive()
qsh2.set_null(np.nan)
qsh2.clip_values(0.1,0.6)
qsh2.set_normal_positive()


qsh2.filter_number_set(8)
qsh2.dim = 15




In [ ]:

fig = plt.figure('gen_missing_curve',figsize=(18, 6))
plt.clf()
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)    
# ax1.set_xlim(-2.,2.)
ax2.set_ylim(0.,1.5)

qsh2.shuffle()
for xy in qsh2.ds_array.take(1):
    xy,_ = xy
    print(xy)
    x,y = tf.split(xy,2)
    ax2.scatter(x,y,s=80)
    m,v  = q_vae.encode([xy])    
    gpt = m[0].numpy()
    print(gpt)
    ax1.scatter(gpt[0],gpt[1])

    XY = q_vae.decode(m,apply_sigmoid=True)
    X,Y = tf.split(XY[0], 2)
    X,Y = (X.numpy(), Y.numpy())

    ax2.scatter(X,Y,s=40)



# CLEAN DATABASE

In [ ]:
## CLEAN QSH DATABASE
!pip install tqdm --user
from tqdm import tqdm
import copy
qsh_clean = copy.deepcopy(qsh)

ds = qsh.ds_array
for i,X in enumerate(tqdm(ds.batch(1), total=len(qsh))):
    X,_ = X
    s   = int(q_vae.feature_dim/2)
    Xr  = q_vae.recover(X)
    x,y = tf.split(Xr, 2, axis=1)
    qsh_clean[i].prel[:s] = x
    qsh_clean[i].te[:s]   = y

    


In [ ]:
# qsh_clean.save(ipysh.abs_builddir+'/te_db_r15_clean.npy')

In [ ]:
import copy
qsh_clean = copy.deepcopy(qsh)
qsh_clean.load(ipysh.abs_builddir+'/te_db_r15_clean.npy')
qsh_clean.dim = 15

# CONVOLUTIONAL VAE

In [ ]:
cae = models.CAEFIT.CAEFIT(latent_dim=2, feature_dim=30, dprate=0.4, scale=2, beta=0.)

# im1 = tf.keras.utils.plot_model(cae.inference_net, show_shapes=True, rankdir='LR')
# im2 = tf.keras.utils.plot_model(cae.generative_net, show_shapes=True, rankdir='LR')
# from IPython.display import display
# display(im1)
# display(im2)

In [ ]:
cae = models.CAEFIT1.CAEFIT1(latent_dim=2, feature_dim=30, dprate=0.4, scale=2, beta=0.)

In [ ]:
# ipysh.Bootstrap_support.debug()

p = Hplt.LSPlotBokeh()
p.set_model(cae)
p.set_data(qsh_clean, counts=1000)
p.plot_notebook(notebook_url='http://172.17.0.2:8888')

In [ ]:
ds = qsh_clean.ds_array
ds0,_ = [x for x in ds.take(1).batch(1)][0]
ds0
me,va = cae.encode(ds0)
y = cae.decode(me, apply_sigmoid=True)
y

In [ ]:
# train(cae, qsh_clean, batch=200, epoch=10, loss_factor=1e-4)
# models.base.train(cae, qsh_clean, batch=200, epoch=2, loss_factor=1e-3)
models.base.train(cae, qsh_clean, batch=200, epoch=2, loss_factor=1e-3)

In [ ]:
models.base.train_thread(cae, qsh_clean, batch=200, epoch=5, loss_factor=1e-3).start()

In [ ]:
# cae.save('step7_cae_r15')

In [ ]:
cae.load('step7_cae_r15')

In [ ]:
qsh_pos = 0

In [ ]:
fig = plt.figure('test_curve',figsize=(18, 6))
plt.clf()
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)    
# ax1.set_xlim(-2.,2.)
ax2.set_ylim(0.,1.)


for xy in qsh_clean.ds_array.skip(qsh_pos).take(1):    
    xy,_ = xy
    x,y = tf.split(xy,2)
    ax2.scatter(x,y,s=80)
    m,v  = cae.encode([xy], training=False)
    gpt = m[0].numpy()
    ax1.scatter(gpt[0],gpt[1])
    
    XY = cae.decode(m,apply_sigmoid=True, training=False)
    X,Y = tf.split(XY[0], 2)
    X,Y = (X.numpy(), Y.numpy())
    ax2.scatter(X,Y,s=40)

print(qsh_pos)    
qsh_pos += 1


# GAN on qsh_clean

In [ ]:
!pip install --user nbmultitask
from nbmultitask import ThreadWithLogAndControls
from nbmultitask import ProcessWithLogAndControls
from IPython.display import clear_output

In [ ]:
%aimport models.GAN0
from models.GAN0 import GAN0
gan = GAN0(latent_dim=2, feature_dim=30, dprate=0.1, scale=1)

In [ ]:
class AsyncTrain(ThreadWithLogAndControls):
    def __init__(self, model, data, **kwargs):
        self._model = model
        model.stop_training = False
        fn = lambda thread_print: model.train(data, **kwargs)
        super(AsyncTrain, self).__init__(target=fn, name='async train')
    
    def terminate(self):
        self._model.stop_training = True

task = AsyncTrain(gan, qsh_clean, epoch=50, batch=50, optimizer_factor=1e-3)
task.control_panel()

In [ ]:
from bokeh.io import show, output_notebook
output_notebook()

p = Hplt.LSPlotBokeh()
p.set_model(gan)
p.set_data(qsh_clean, counts=1000)
p.plot(notebook_url='http://172.17.0.2:8888')